# データフレームの操作

Julia で引用できるデータを例として，データフレームの操作法について述べる。

In [2]:
using RDatasets

RDatasets.dataset("datasets", "airquality") のように，第 2 引数で指定したデータセットが利用できる。

In [7]:
airquality = RDatasets.dataset("datasets", "airquality");

## データフレームの概要

### データセットの大きさ

データセットの大きさは size() で得られる。結果はタプル (行数, 列数) で返される。

In [99]:
size(airquality)

(153, 6)

### データフレームの表示

データフレームの最初の n 行，最後の m 行を表示するには，first(df, n), last(df, m) のように指定する。
n, m が省略された場合は 1 が仮定される。

In [100]:
first(airquality, 5)

,Ozone,Solar.R,Wind,Temp,Month,Day
,Int64?,Int64?,Float64,Int64,Int64,Int64
1,41,190,7.4,67,5,1
2,36,118,8.0,72,5,2
3,12,149,12.6,74,5,3
4,18,313,11.5,62,5,4
5,missing,missing,14.3,56,5,5


In [101]:
last(airquality, 3)

,Ozone,Solar.R,Wind,Temp,Month,Day
,Int64?,Int64?,Float64,Int64,Int64,Int64
1,14,191,14.3,75,9,28
2,18,131,8.0,76,9,29
3,20,223,11.5,68,9,30


データフレームの表示の 1 行目は，列名（変数名），2 行目はそれぞれのデータの型である。
Int64 は整数型，Float64 は実数型，String は文字列型である。なお，それぞれの型名の後に ? がついている場合は，データ中に欠損値 missing が含まれることを表している。

### データフレームの変数名

変数名は names() で得られる。

In [102]:
names(airquality)

6-element Vector{String}:
 "Ozone"
 "Solar.R"
 "Wind"
 "Temp"
 "Month"
 "Day"

### データフレームの列の参照

Julia では，列を参照するときに何通りかの方法がある。

いかの方法は，1 つの列全体を参照する。
列名の前に : を付けたもの，以下の例では :Ozone は Julia では「シンボル」と呼ばれる（必ずしもデータフレームの取り扱いのときだけでなく，整数や文字列のようなものとして扱われる）。

In [8]:
airquality.Ozone;
airquality."Ozone"
airquality[:, :Ozone];
airquality[:, "Ozone"];
firstcolumn = :Ozone
airquality[:, firstcolumn];
airquality[:, 1];

1 番目の指定法が一番わかり易いと思うが，例からも想像できるように，変数名の中に '.' があるとエラーになる。そのような場合には前もって列名を変更するか 2 番目の方法すなわち引用符でくくればよい。

In [104]:
# airquality.Solar.R
# ArgumentError: column name :Solar not found in the data frame; existing most similar names are: :Solar.R

In [ ]:
airquality."Solar.R";

列を範囲でで参照するときには [第 1 引数, 第 2 引数] のように指定する。第 1 引数は行，第 2 引数は列を指定する。

「数1:数2」のように表したときは数1から数2までの整数，[数1, 数2, ...]　あるいは [シンボル1, シンボル2, ...]
 あるいは [文字列1, 文字列2, ...] のように表した場合はそれぞれをまとめたもの（集合）として扱われる。特に，: 一文字が使われる場合はその行や列全部を表す。

In [12]:
airquality[1:5, [1, 3]]

,Ozone,Wind
,Int64?,Float64
1,41,7.4
2,36,8.0
3,12,12.6
4,18,11.5
5,missing,14.3


第 1 引数は : で表す他に ! で表すこともある。: はデータフレームをコピーして参照するが，! は直接参照する。

下の 2 つの例において，test の第 1 列は元のままの Int64? であるが，test2 の第1列は，test2 の第2列と同じ Float64 になる。
この違いは，大したことではないこともあるが，大きな落とし穴になることもあるので注意が必要である。

たとえば，下の例で 10 倍する代わりに 5 倍するようにすると。test[:, 1] は 「InexactError: Int64(57.5)」というエラーを発生する。つまり，「元のままの Int64? に 57.5 の整数部 Int(57.5) を大入試要素敦賀精度が失われるよ」というエラーである。

大雑把な方針としては，左辺のデータフレームには ! を使うほうが自然かもしれない。

In [33]:
test = airquality[1:5, [1, 3]]
test[:, 1] = test[:, 2] .* 10
test

,Ozone,Wind
,Int64?,Float64
1,74,7.4
2,80,8.0
3,126,12.6
4,115,11.5
5,143,14.3


In [34]:
test2 = airquality[1:5, [1, 3]]
test2[!, 1] = test2[:, 2] .* 5
test2

,Ozone,Wind
,Float64,Float64
1,37.0,7.4
2,40.0,8.0
3,63.0,12.6
4,57.5,11.5
5,71.5,14.3


### データフレームの列名の変更

列名の変更は rename!() による。
関数名の最後が ! になっている場合は，関数への引数が直接変形されるので代入の必要はない。
一方で，元の引数が変更されてしまうので，前もってコピーをとっておかない限りもとに戻せなくなるので注意が必要である。

In [35]:
airquality2 = rename(airquality, Dict("Solar.R" => "Solar_R"));
first(airquality2)

,Ozone,Solar_R,Wind,Temp,Month,Day
,Int64?,Int64?,Float64,Int64,Int64,Int64
1,41,190,7.4,67,5,1


rename() の場合は，引数として指定した元のデータフレームは変更されない。したがって，もし変更結果を保存したい場合はデータフレーム（自分自身でもよい）に保存しなければならない

In [38]:
first(airquality)

,Ozone,Solar_R,Wind,Temp,Month,Day
,Int64?,Int64?,Float64,Int64,Int64,Int64
1,41,190,7.4,67,5,1


rename!() では，引数として指定した元のデータフレームが変更される。

In [39]:
rename!(airquality, Dict("Solar.R" => "Solar_R"));
first(airquality)

LoadError: ArgumentError: Tried renaming :Solar.R to :Solar_R, when :Solar.R does not exist in the data frame.

### 列の抽出

select()，select!() は列の抽出を行う。抽出した後の順序は指定したとおりになる。
列の指定は何通りかある。

In [41]:
df = select(airquality, [4, 3, 2, 1]); # 列番号で指定
first(df)

,Temp,Wind,Solar_R,Ozone
,Int64,Float64,Int64?,Int64?
1,67,7.4,190,41


In [42]:
df = select(airquality, [:Ozone, :Solar_R, :Wind, :Temp]); # シンボルで指定
first(df)

,Ozone,Solar_R,Wind,Temp
,Int64?,Int64?,Float64,Int64
1,41,190,7.4,67


In [ ]:
列を（名前を変えて）コピーしたり，変数変換することもできる。ByRow() の引数は関数，無名関数である。

元の Temp を Fahrenheit，Temp から変換して Celcius を作成する。

In [127]:
df = select(airquality, :Ozone, :Solar_R, :Wind,
        :Temp => :Fahrenheit, :Temp => ByRow(x -> 5(x-32)/9) => :Celcius);
first(df, 5)

,Ozone,Solar_R,Wind,Fahrenheit,Celcius
,Int64?,Int64?,Float64,Int64,Float64
1,41,190,7.4,67,19.4444
2,36,118,8.0,72,22.2222
3,12,149,12.6,74,23.3333
4,18,313,11.5,62,16.6667
5,missing,missing,14.3,56,13.3333


### グループごとの記述統計量

データフレームをグループを表す変数でグループ化し，それぞれのサブグループごとに統計量を求める。

airquality データセットの :Month でグループ化する。

In [11]:
using RDatasets

airquality = RDatasets.dataset("datasets", "airquality");
select(airquality, Not(:Day));
gdf = groupby(select(airquality, 1:5), :Month);
println("number of subgroup = ", length(gdf))

number of subgroup = 5


gdf[1] は :Month が 5 の場合のサブグループである。

それぞれの gdf[i] は，データフレームと同じように扱うことができる。

In [12]:
describe(gdf[1])

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,Type
1,Ozone,23.6154,1,18.0,115,5,"Union{Missing, Int64}"
2,Solar.R,181.296,8,194.0,334,4,"Union{Missing, Int64}"
3,Wind,11.6226,5.7,11.5,20.1,0,Float64
4,Temp,65.5484,56,66.0,81,0,Int64
5,Month,5.0,5,5.0,5,0,Int64


for ループで，すべてのサブグループについて分析できる。

In [17]:
for df in gdf
    println("\nMonth: ", df[1, :Month], "\n", describe(df))
end


Month: 5
5×7 DataFrame
 Row │ variable  mean      min   median   max    nmissing  eltype                
     │ Symbol    Float64   Real  Float64  Real   Int64     Type                  
─────┼───────────────────────────────────────────────────────────────────────────
   1 │ Ozone      23.6154   1       18.0  115           5  Union{Missing, Int64}
   2 │ Solar.R   181.296    8      194.0  334           4  Union{Missing, Int64}
   3 │ Wind       11.6226   5.7     11.5   20.1         0  Float64
   4 │ Temp       65.5484  56       66.0   81           0  Int64
   5 │ Month       5.0      5        5.0    5           0  Int64

Month: 6
5×7 DataFrame
 Row │ variable  mean      min   median   max    nmissing  eltype                
     │ Symbol    Float64   Real  Float64  Real   Int64     Type                  
─────┼───────────────────────────────────────────────────────────────────────────
   1 │ Ozone      29.4444  12       23.0   71          21  Union{Missing, Int64}
   2 │ Solar.R   190

変数ごとに一覧表示するには，プログラムを書く必要がある。

In [18]:
a = [describe(df) for df in gdf]
variable = ["Ozone", "Solar.R", "Wind", "Temp"]
for i in variable
    df = DataFrame[]
    for j in 1:length(a)
        pos = indexin([Symbol(i)], a[j][:, 1])[1]
        ismissing(pos) && continue
        row = a[j][pos, :]

5-element Vector{DataFrame}:
 5×7 DataFrame
 Row │ variable  mean      min   median   max    nmissing  eltype              ⋯
     │ Symbol    Float64   Real  Float64  Real   Int64     Type                ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Ozone      23.6154   1       18.0  115           5  Union{Missing, Int6 ⋯
   2 │ Solar.R   181.296    8      194.0  334           4  Union{Missing, Int6
   3 │ Wind       11.6226   5.7     11.5   20.1         0  Float64
   4 │ Temp       65.5484  56       66.0   81           0  Int64
   5 │ Month       5.0      5        5.0    5           0  Int64               ⋯
                                                                1 column omitted
 5×7 DataFrame
 Row │ variable  mean      min   median   max    nmissing  eltype              ⋯
     │ Symbol    Float64   Real  Float64  Real   Int64     Type                ⋯
─────┼──────────────────────────────────────────────────────────────────────────
 

In [8]:
using DataFrames, Statistics
for variable in ["Ozone", "Solar.R", "Wind", "Temp"]
    println("\n$variable")
    result = DataFrame[]
    for i in 1:length(gdf)
        each = dropmissing(select(gdf[i], ["Month", variable])) # nrow(each)
        res = combine(each, :Month => first => :Month, nrow => :n, variable => mean => :Mean, variable => std => :SD)
        if result == []
            result = res
        else
            result = vcat(result, res)
        end
    end
    println(result)
end


Ozone
5×4 DataFrame
 Row │ Month  n      Mean     SD      
     │ Int64  Int64  Float64  Float64 
─────┼────────────────────────────────
   1 │     5     26  23.6154  22.2244
   2 │     6      9  29.4444  18.2079
   3 │     7     26  59.1154  31.6358
   4 │     8     26  59.9615  39.6812
   5 │     9     29  31.4483  24.1418

Solar.R
5×4 DataFrame
 Row │ Month  n      Mean     SD       
     │ Int64  Int64  Float64  Float64  
─────┼─────────────────────────────────
   1 │     5     27  181.296  115.075
   2 │     6     30  190.167   92.883
   3 │     7     31  216.484   80.5683
   4 │     8     28  171.857   76.8349
   5 │     9     30  167.433   79.1183

Wind
5×4 DataFrame
 Row │ Month  n      Mean      SD      
     │ Int64  Int64  Float64   Float64 
─────┼─────────────────────────────────
   1 │     5     31  11.6226   3.53145
   2 │     6     30  10.2667   3.76923
   3 │     7     31   8.94194  3.03598
   4 │     8     31   8.79355  3.22593
   5 │     9     30  10.18     3.46125



In [ ]:
attitude.csv.gz
airquality.csv.gz
anscombe.csv.gz
attenu.csv.gz
BOD.csv.gz
cars.csv.gz
chickwts.rda
CO2.rda
crimtab.csv.gz
esoph.rda
euro.csv.gz
faithful.csv.gz
Formaldehyde.csv.gz
freeny.csv.gz
HairEyeColor.csv.gz
infert.rda
InsectSprays.rda
iris.rda
islands.csv.gz
LifeCycleSavings.csv.gz
Loblolly.rda
longley.csv.gz
morley.csv.gz
mtcars.csv.gz
occupationalStatus.csv.gz
OrchardSprays.rda
PlantGrowth.rda
precip.csv.gz
pressure.csv.gz
Puromycin.rda
quakes.csv.gz
randu.csv.gz
rivers.csv.gz
rock.csv.gz
sleep.rda
stackloss.csv.gz
swiss.csv.gz
Theoph.rda
Titanic.csv.gz
ToothGrowth.rda
trees.csv.gz
UCBAdmissions.csv.gz
USArrests.csv.gz
USJudgeRatings.csv.gz
USPersonalExpenditure.csv.gz
VADeaths.csv.gz
volcano.csv.gz
warpbreaks.rda
women.csv.gz
WorldPhones.csv.gz